In [ ]:
import pandas as pd
import numpy as np

file_path = '/home/asd/projects/seoul_metro_congestion/seoul_metro_congestion.csv'

try:
    df = pd.read_csv(file_path, encoding='cp949')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='utf-8')



In [16]:

# 데이터의 첫 5줄 확인
print("--- 원본 데이터 (처음 5줄) ---")
print(df.head())


--- 원본 데이터 (처음 5줄) ---
  요일구분   호선  역번호  출발역 상하구분  5시30분  6시00분  6시30분  7시00분  7시30분  ...  20시00분  \
0   평일  1호선  150  서울역   상선    8.3   21.8   22.2   39.0   64.4  ...    19.3   
1   평일  1호선  150  서울역   하선   11.5   11.3   16.3   24.9   38.1  ...    37.4   
2   평일  1호선  151   시청   상선    7.8   15.2   18.8   30.4   49.1  ...    23.1   
3   평일  1호선  151   시청   하선    9.4   10.3   17.2   23.0   36.3  ...    33.8   
4   평일  1호선  152   종각   상선    7.1   12.0   11.6   18.8   27.0  ...    30.5   

   20시30분  21시00분  21시30분  22시00분  22시30분  23시00분  23시30분  00시00분  00시30분  
0    18.3    18.8    16.7    19.3    19.7    13.6    12.6     5.7     1.3  
1    36.5    40.0    35.6    36.0    28.5    17.2    14.2     7.7     0.0  
2    23.1    23.3    19.9    23.5    22.6    18.3    14.2     6.2     1.2  
3    34.0    38.2    30.2    31.5    24.0    15.4    12.4     5.2     0.2  
4    30.4    32.1    25.3    29.6    29.7    24.9    16.3     6.9     1.1  

[5 rows x 44 columns]


In [17]:
# 데이터의 전체 구조 및 타입 확인
print("\n--- 원본 데이터 정보 ---")
df.info()


--- 원본 데이터 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1671 entries, 0 to 1670
Data columns (total 44 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    1671 non-null   object 
 1   호선      1671 non-null   object 
 2   역번호     1671 non-null   int64  
 3   출발역     1671 non-null   object 
 4   상하구분    1671 non-null   object 
 5   5시30분   1671 non-null   float64
 6   6시00분   1671 non-null   float64
 7   6시30분   1671 non-null   float64
 8   7시00분   1671 non-null   float64
 9   7시30분   1671 non-null   float64
 10  8시00분   1671 non-null   float64
 11  8시30분   1671 non-null   float64
 12  9시00분   1671 non-null   float64
 13  9시30분   1671 non-null   float64
 14  10시00분  1671 non-null   float64
 15  10시30분  1671 non-null   float64
 16  11시00분  1671 non-null   float64
 17  11시30분  1671 non-null   float64
 18  12시00분  1671 non-null   float64
 19  12시30분  1671 non-null   float64
 20  13시00분  1671 non-null   float64
 21  13시30분  1671 non-n

In [18]:
# 'id_vars'는 고정할 기준 컬럼들입니다. (데이터에 따라 '연번', '역번호' 등이 더 있을 수 있음)
# 실제 데이터의 컬럼명을 확인하고 정확히 지정해야 합니다.
id_columns = ['요일구분', '호선', '역번호', '출발역', '상하구분'] 

# id_vars에 포함되지 않은 나머지 컬럼 (즉, 시간대 컬럼들)을 모두 melt 대상으로 삼습니다.
# value_vars를 명시적으로 지정하지 않으면 id_vars를 제외한 모든 컬럼이 대상이 됩니다.
df_long = pd.melt(df, 
                  id_vars=id_columns, 
                  var_name='시간대', 
                  value_name='혼잡도')

print("\n--- Long Format으로 변경된 데이터 (처음 5줄) ---")
print(df_long.head())


--- Long Format으로 변경된 데이터 (처음 5줄) ---
  요일구분   호선  역번호  출발역 상하구분    시간대   혼잡도
0   평일  1호선  150  서울역   상선  5시30분   8.3
1   평일  1호선  150  서울역   하선  5시30분  11.5
2   평일  1호선  151   시청   상선  5시30분   7.8
3   평일  1호선  151   시청   하선  5시30분   9.4
4   평일  1호선  152   종각   상선  5시30분   7.1


In [19]:
df_long['혼잡도'] = df_long['혼잡도'].fillna(0)


print("\n--- '혼잡도' 컬럼 정제 후 (처음 5줄) ---")
print(df_long.head())


--- '혼잡도' 컬럼 정제 후 (처음 5줄) ---
  요일구분   호선  역번호  출발역 상하구분    시간대   혼잡도
0   평일  1호선  150  서울역   상선  5시30분   8.3
1   평일  1호선  150  서울역   하선  5시30분  11.5
2   평일  1호선  151   시청   상선  5시30분   7.8
3   평일  1호선  151   시청   하선  5시30분   9.4
4   평일  1호선  152   종각   상선  5시30분   7.1


In [28]:
# (1) '시 ' 또는 '시' -> ':' (e.g., '05시 30분' -> '05:30분')
temp_time = df_long['시간대'].str.replace('시 ', ':', regex=False)
temp_time = temp_time.str.replace('시', ':', regex=False)

# (2) '분' -> '' (e.g., '05:30분' -> '05:30')
temp_time = temp_time.str.replace('분', '', regex=False)

# 2. '-'를 기준으로 분리하여 시작 시간(e.g., '05:30')만 추출
#    (e.g., '05:30-05:59' -> '05:30')
df_long['시간'] = temp_time.str.split('-').str[0]

# 3. (혹시 모를) 앞뒤 공백 제거
df_long['시간'] = df_long['시간'].str.strip()


# 4. '주중/주말' 컬럼 생성
df_long['주중주말'] = np.where(df_long['요일구분'] == '평일', '주중', '주말')


print("\n--- 파생 변수 생성 후 (시간 포맷 변경 확인) ---")
# '시간대'(원본), '시간'(변경 후)을 함께 출력하여 잘 바뀌었는지 확인
print(df_long[['시간대', '시간', '주중주말']].head())

print("\n--- 최종 데이터 정보 ---")
df_long.info()



--- 파생 변수 생성 후 (시간 포맷 변경 확인) ---
     시간대    시간 주중주말
0  5시30분  5:30   주중
1  5시30분  5:30   주중
2  5시30분  5:30   주중
3  5시30분  5:30   주중
4  5시30분  5:30   주중

--- 최종 데이터 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65169 entries, 0 to 65168
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    65169 non-null  object 
 1   호선      65169 non-null  object 
 2   역번호     65169 non-null  int64  
 3   출발역     65169 non-null  object 
 4   상하구분    65169 non-null  object 
 5   시간대     65169 non-null  object 
 6   혼잡도     65169 non-null  float64
 7   시간      65169 non-null  object 
 8   주중주말    65169 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 4.5+ MB


In [29]:
# 1. (가장 중요) 현재 '호선' 컬럼에 어떤 값들이 있는지 확인합니다.
#    (출력된 리스트를 보고 2번 딕셔너리를 만드세요)
current_lines = df_long['호선'].unique()
print("--- 현재 호선 이름 ---")
print(current_lines)

--- 현재 호선 이름 ---
['Line 1' 'Line 2' 'Line 3' 'Line 4' 'Line 5' 'Line 6' 'Line 7' 'Line 8']


In [30]:
# 2. 한글 이름을 영어 이름으로 바꾸는 '규칙' (Dictionary)을 만듭니다.
line_mapping = {
    '1호선': 'Line 1',
    '2호선': 'Line 2',
    '3호선': 'Line 3',
    '4호선': 'Line 4',
    '5호선': 'Line 5',
    '6호선': 'Line 6',
    '7호선': 'Line 7',
    '8호선': 'Line 8',
    '9호선': 'Line 9',
}

df_long['호선'] = df_long['호선'].replace(line_mapping)

print("\n--- 변경 후 호선 이름 ---")
print(df_long['호선'].unique())

print("\n'호선' 컬럼 영문 변경 완료!")


--- 변경 후 호선 이름 ---
['Line 1' 'Line 2' 'Line 3' 'Line 4' 'Line 5' 'Line 6' 'Line 7' 'Line 8']

'호선' 컬럼 영문 변경 완료!


In [ ]:
# 전처리 완료된 데이터를 새 파일로 저장
df_long.to_csv('preprocessed_congestion.csv', index=False, encoding='utf-8-sig')

print("\n--- 전처리 완료된 파일 저장 성공! ---")


--- 전처리 완료된 파일 저장 성공! ---
